In [37]:
import os
import numpy as np
from utils import *
import pandas as pd

class config:
    data_path = '../../../datasets/garanti-bbva-data-camp/'

<IPython.core.display.Javascript object>

In [38]:
train_df = pd.read_csv(os.path.join(config.data_path, 'train_users.csv'))
test_df = pd.read_csv(os.path.join(config.data_path, 'test_users.csv'))
submission = pd.read_csv(os.path.join(config.data_path, 'submission.csv'))
skills = pd.read_csv(os.path.join(config.data_path, 'skills.csv')) 
work_experiences = pd.read_csv(os.path.join(config.data_path, 'work_experiences.csv')) 
languages = pd.read_csv(os.path.join(config.data_path, 'languages.csv'))
education = pd.read_csv(os.path.join(config.data_path, 'education.csv'))

print(f'train_df shape: {train_df.shape}')
print(f'test_df shape: {test_df.shape}')
print(f'submission shape: {submission.shape}')
print(f'skills shape: {skills.shape}')
print(f'work_experiences shape: {work_experiences.shape}')
print(f'education shape: {education.shape}')
print(f'languages shape: {languages.shape}')

train_df shape: (53019, 4)
test_df shape: (13255, 3)
submission shape: (13255, 2)
skills shape: (1398443, 2)
work_experiences shape: (187019, 4)
education shape: (142575, 6)
languages shape: (76062, 3)


<IPython.core.display.Javascript object>

In [39]:
train_df.head()

,user_id,industry,location,moved_after_2019
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1
1,6950,Internet,"Istanbul, Istanbul, Turkey",0
2,4880,Online Media,Turkey,0
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0
4,11005,Banking,"Istanbul, Turkey",0


<IPython.core.display.Javascript object>

In [40]:
test_df.head()

,user_id,industry,location
0,17449,Research,Turkey
1,33967,Computer Software,"Istanbul, Istanbul, Turkey"
2,2110,Automotive,Turkey
3,55082,Internet,Turkey
4,37165,Electrical/Electronic Manufacturing,Turkey


<IPython.core.display.Javascript object>